In [81]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import feather

In [2]:
data = pd.read_csv("Datasets/Round_1_preprocessing/preproccesed_outliers_1250895_col_12_threshold_08.csv")
data.head()

,Unnamed: 0,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
1,2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
2,3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
3,4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day
4,5,A-6,MapQuest,201.0,3,2016-02-08 07:44:26,2016-02-08 08:14:26,40.100590,-82.925194,0.01,...,False,False,False,False,False,False,Day,Day,Day,Day


In [67]:
N = 10000    #number of instances we want to use

drop = ['Unnamed: 0', 'ID', 'Start_Time', 'End_Time', 'Weather_Timestamp', 'Source', 'Description', 'Turning_Loop', 'Bump', 'No_Exit', 'Give_Way', 'Traffic_Calming']
data_feather = data.drop(drop, axis=1).dropna().iloc[:N, :].reset_index(drop=True)
data_feather.to_feather("Datasets/Round_1_preprocessing/preproccesed_outliers_1250895_col_12_threshold_08.feather")
data_feather = feather.read_dataframe("Datasets/Round_1_preprocessing/preproccesed_outliers_1250895_col_12_threshold_08.feather")

In [69]:
# Create one hot encoding labels
n = len(data_feather)
labels = data_feather['Severity']
labels_oh = np.zeros((n, 4))
for i in range(n):
    labels_oh[i, labels[i]-1] = 1
    
# Create one hot encoding attributes
atts_oh = OneHotEncoder().fit_transform(data_feather.drop('Severity', axis=1)).toarray()

In [82]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(atts_oh, labels_oh, test_size=0.33, random_state=42)

In [89]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(30, activation=tf.nn.swish, input_shape=(24445,))) # Swish indeed performs better than sigmoid
model.add(tf.keras.layers.Dense(4, activation=tf.nn.softmax))

# Compile and print out summary of model
model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop',
             metrics=['accuracy'])

model.summary()

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 30)                733380    
_________________________________________________________________
dense_23 (Dense)             (None, 4)                 124       
Total params: 733,504
Trainable params: 733,504
Non-trainable params: 0
_________________________________________________________________


In [90]:
EPOCHS = 10
BATCH_SIZE = 128

In [91]:
# Train the model
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/10
6700/6700 [==============================] - 2s 239us/sample - loss: 0.6875 - acc: 0.7157
Epoch 2/10
6700/6700 [==============================] - 1s 206us/sample - loss: 0.3492 - acc: 0.8830
Epoch 3/10
6700/6700 [==============================] - 1s 208us/sample - loss: 0.2449 - acc: 0.9182
Epoch 4/10
6700/6700 [==============================] - 1s 210us/sample - loss: 0.1886 - acc: 0.9375
Epoch 5/10
6700/6700 [==============================] - 1s 208us/sample - loss: 0.1503 - acc: 0.9536
Epoch 6/10
6700/6700 [==============================] - 1s 209us/sample - loss: 0.1235 - acc: 0.9619
Epoch 7/10
6700/6700 [==============================] - 1s 209us/sample - loss: 0.1022 - acc: 0.9687
Epoch 8/10
6700/6700 [==============================] - 1s 209us/sample - loss: 0.0866 - acc: 0.9737
Epoch 9/10
6700/6700 [==============================] - 1s 209us/sample - loss: 0.0733 - acc: 0.9782
Epoch 10/10
6700/6700 [==============================] - 1s 208us/sample - loss: 0.0633 - a

In [92]:
# Testing the model 
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\ntest_acc: ', test_acc)

3300/3300 [==============================] - 1s 220us/sample - loss: 0.3043 - acc: 0.9036

test_acc:  0.90363634
